# How to Build an ACS Lookup Table
The purpose of this notebook is to illustrate how to build an American Community Survey (ACS) lookup table at the census tract level. Note ACS data has been reduced, table names can be supplied to the required tables to change what data is returned in the lookup table. 

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser
import datetime as dt
import pandas as pd
import numpy as np
import warnings
import glob
import json
import sys
import os
import re

In [3]:
warnings.filterwarnings(action='once')
home = expanduser('~')

In [4]:
src_path = '{}/zest-race-predictor/playground/kam/zrp'.format(home)
sys.path.append(src_path)

Predefine paths & required parameters

In [5]:
# Support files path pointing to where the raw ACS data is stored
support_files_path = "/d/shared/zrp/shared_data"
# Year of ACS data
year = "2019"
# Span of ACS data. The ACS data is available in 1 or 5 year spans. 
span = "5"
# Geo level to build lookup table at
geo='tract'
n_jobs = -1

In [6]:
from prepare.acs_lookup import *

### Run `ACS_LookupBuilder`
This class constructs American Community Survey lookup tables that enables race approximation.

In [7]:
%%time 
acs_build = ACS_LookupBuilder(support_files_path = support_files_path, year = year, geo = geo,span = span, n_jobs =n_jobs)
output = acs_build.transform(False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 60 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:   19.7s
[Parallel(n_jobs=-1)]: Done 101 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done 193 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed:   23.9s
[Parallel(n_jobs=-1)]: Done 245 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   24.8s
[Parallel(n_jobs=-1)]: Done 301 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 330 tasks      | elapsed:  

  Table: B01003
     (74001, 1)
  Table: B02001
     (74001, 10)
  Table: B03001
     (74001, 31)
  Table: B04004
     (74001, 108)
  Table: B04006
     (74001, 116)
  Table: B04007
  Table: B05011
     (74001, 16)
  Table: B05012
  Table: B06009
     (74001, 60)
  Table: B07101
     (74001, 60)
  Table: B08301
     (74001, 21)
  Table: B10051
     (74001, 74)
  Table: B11017
     (74001, 3)
  Table: B16001
     (74001, 166)
  Table: B19001
     (74001, 170)
  Table: B23020
     (74001, 3)
  Table: B25004
     (74001, 8)
  Table: B25075
     (74001, 27)
  Table: B99021
     (74001, 3)
  Table: B99162
     (74001, 7)
  Table: C16001

No tables were saved
CPU times: user 44.5 s, sys: 6.28 s, total: 50.8 s
Wall time: 3min 47s


### Preview 

In [8]:
output.shape

(74001, 884)

In [9]:
output.head()

,,,B01003_001,B02001_001,B02001_002,B02001_003,B02001_004,B02001_005,B02001_006,B02001_007,B02001_008,B02001_009,...,B99021_001,B99021_002,B99021_003,B99162_001,B99162_002,B99162_003,B99162_004,B99162_005,B99162_006,B99162_007
GEOID,GEO_NAME,EXT_GEOID,,,,,,,,,,,,,,,,,,,,,
01001020100,"Census Tract 201, Autauga County, Alabama",14000US01001020100,1993,1993,1685,152,0,2,0,0,154,0,...,1993,8,1985,1878,1814,64,6,0,6,58
01001020200,"Census Tract 202, Autauga County, Alabama",14000US01001020200,1959,1959,759,1117,0,0,21,6,56,0,...,1959,37,1922,1860,1846,14,0,0,0,14
01001020300,"Census Tract 203, Autauga County, Alabama",14000US01001020300,3507,3507,2285,857,0,32,5,233,95,0,...,3507,256,3251,3410,3088,322,0,0,0,322
01001020400,"Census Tract 204, Autauga County, Alabama",14000US01001020400,3878,3878,3495,216,32,11,0,0,124,14,...,3878,50,3828,3716,3512,204,16,0,16,188
01001020500,"Census Tract 205, Autauga County, Alabama",14000US01001020500,10596,10596,7982,1929,0,385,0,49,251,0,...,10596,131,10465,10091,9118,973,37,23,14,936


In [10]:
output.tail()

,,,B01003_001,B02001_001,B02001_002,B02001_003,B02001_004,B02001_005,B02001_006,B02001_007,B02001_008,B02001_009,...,B99021_001,B99021_002,B99021_003,B99162_001,B99162_002,B99162_003,B99162_004,B99162_005,B99162_006,B99162_007
GEOID,GEO_NAME,EXT_GEOID,,,,,,,,,,,,,,,,,,,,,
72153750501,"Census Tract 7505.01, Yauco Municipio, Puerto Rico",14000US72153750501,6113,6113,4907,145,80,0,0,912,69,61,...,6113,340,5773,5904,578,5326,338,338,0,4988
72153750502,"Census Tract 7505.02, Yauco Municipio, Puerto Rico",14000US72153750502,2388,2388,1763,201,21,0,0,374,29,25,...,2388,70,2318,2223,235,1988,228,228,0,1760
72153750503,"Census Tract 7505.03, Yauco Municipio, Puerto Rico",14000US72153750503,2126,2126,1525,86,0,6,0,475,34,12,...,2126,17,2109,2042,90,1952,83,77,6,1869
72153750601,"Census Tract 7506.01, Yauco Municipio, Puerto Rico",14000US72153750601,4397,4397,3203,316,51,0,0,798,29,29,...,4397,32,4365,4036,396,3640,220,193,27,3420
72153750602,"Census Tract 7506.02, Yauco Municipio, Puerto Rico",14000US72153750602,2225,2225,1436,23,0,0,0,742,24,8,...,2225,131,2094,2114,118,1996,246,246,0,1750
